In [1]:
import numpy as np
from pyscf import gto, scf, fci

In [2]:
# This function is just here for test purposes, for details see 1RDMFT.ibypn
def ONERDMFT_hartree_energy(Fouridx, C, n, mol):
    energy = 0
    M = Fouridx.shape[0]
    N = mol.nelec[0]
    for a in  range(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu,nu,kappa,lamda]

    return energy


In [3]:
mol = gto.Mole()
mol.atom = """
    He    0.    0.    0.
"""
# this basis has 2 functions for Helium
mol.basis = "6-31g" #
mol.basis = "ccpvdz"
#, mol.basis = "sto-6g"
mol.build()

# the 2 electron integrals \langle \mu \nu | \kappa \lambda \rangle have M^4 entries
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')


## Run Hartree-Fock.
mf = scf.RHF(mol)
mf.kernel()

C = mf.mo_coeff
h = mf.get_hcore()
J = mf.get_j()

N = mol.nelec[0]
P=np.matmul(C[:,0:N],C[:,0:N].T)
print(P)

converged SCF energy = -2.85516047724274
[[0.35119405 0.304028   0.         0.         0.        ]
 [0.304028   0.26319645 0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


To transform the density matrix from AO to MO: 
$$\boldsymbol{\gamma} = \bf{C}^T \bf{S}^{T} \bf{P} \bf{S} \bf{C}$$

In [4]:
# This is the AO - MO transformation of P to gamma (SMH) -> \gamma = C^T S P S C 
gamma =np.matmul(np.matmul(C.T,np.matmul(np.matmul(S,P),S)), C)
print(gamma)

[[ 1.00000000e+00  1.73362861e-16  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [-2.36091398e-17  1.90757071e-16  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00]]


Transforming back to check consistency:
$$\bf{P} = \bf{C} \boldsymbol{\gamma} \bf{C}^T $$

In [8]:
print(np.matmul(C,np.matmul(gamma,C.T)))

[[0.35119405 0.304028   0.         0.         0.        ]
 [0.304028   0.26319645 0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]
 [0.         0.         0.         0.         0.        ]]


The density matrix in the MO basis is the one in textbooks, i.e.:
$$\boldsymbol{\gamma} = \bf{U}^{T} \bf{n} \bf{U}$$

In [6]:
occu, naturalC = np.linalg.eigh(gamma)
#print("Wie man sieht ist P bzw. gama nun in seiner Eigenbasis. Gleichzeitig habe ich nur in die MO basis gewechselt. Das würde bedeuten in Hartree Fock sind die natürliche Orbitale gleich den Hartree Fock Orbitalen.")
print(occu) 

[0.00000000e+00 0.00000000e+00 0.00000000e+00 1.90757071e-16
 1.00000000e+00]


In [7]:
CTT = np.matmul(C,naturalC)

print(ONERDMFT_hartree_energy(eri, CTT, occu, mol), 1/2*np.trace(np.matmul(J,P)))

1.0268646253637668 1.0268646253637652
